<a href="https://colab.research.google.com/github/quartermaine/Machine-Learning-Notebooks/blob/main/Forecasting/Lag_Llama%2BNeural_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This notebook is an extension of the forecasting notebook. The analysis was conducted through the following steps:

#### - Data Preprocessing: The initial step involved aggregating the claim amounts for each day to gain a clearer perspective on the data. Subsequently, I selected the features to be used for fitting the forecasting models.


#### - Model Fitting: I fitted various state-of-the-art models, including LagLlama, NBEATS, NHITS, and LSTM, to identify which models could deliver the best results.


#### - Forecast Visualization: I created visualizations of the forecasts produced by the different models to facilitate a comparative analysis.


#### - Metric Reporting: Finally, I reported metrics to evaluate the performance of the models. These metrics provided valuable insights into the quality and accuracy of the forecasts.

In [ ]:
#@title Libraries installations 1/2
%%capture
!git clone https://github.com/time-series-foundation-models/lag-llama/




In [ ]:
#@title Change dir

%cd /content/lag-llama


/content/lag-llama


In [ ]:
#@title Libraries installations 2/2
%%capture

!echo -e "\ngluonts==0.14.4" >> requirements.txt

!pip install -r requirements.txt --quiet # this could take some time # ignore the errors displayed by colab

!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/lag-llama

!pip install neuralforecast

In [ ]:
#@title Libraries imports

# pandas
import pandas as pd
# numpy
import numpy as np
# itertools
from itertools import islice
#matplotlib
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
#plotly
import plotly.graph_objects as go
import plotly.io as pio

import torch
# lag_llama
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.pandas import PandasDataset
from lag_llama.gluon.estimator import LagLlamaEstimator
# IPython
from IPython.display import display, Markdown
# neuralforecast
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.models import LSTM, NHITS, RNN
# sklearn
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, mean_absolute_error, r2_score


/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [ ]:
#@title Functions definitions

def get_lag_llama_predictions(dataset, prediction_length, device, context_length=128, use_rope_scaling=False, num_samples=100):
    ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    rope_scaling_arguments = {
        "type": "linear",
        "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
    }

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length, # Lag-Llama was trained with a context length of 32, but can work with any context length

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        rope_scaling=rope_scaling_arguments if use_rope_scaling else None,

        batch_size=1,
        num_parallel_samples=100,
        device=device,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

def _get_lag_llama_dataset(dataset):
    # avoid mutations
    dataset = dataset.copy()

    # convert numerical columns to `float32`
    for col in dataset.columns:
        if dataset[col].dtype != "object" and not pd.api.types.is_string_dtype(
            dataset[col]
        ):
            dataset[col] = dataset[col].astype("float32")

    # create a `PandasDataset`
    backtest_dataset = PandasDataset(dict(dataset),
                                     freq='D'
                                     )

    return backtest_dataset


In [ ]:
#@title Import data

# from gluonts.dataset.pandas import PandasDataset

data_path = '/content/insurance_data_sample.csv'

df = pd.read_csv(data_path,
                 parse_dates=True
                 )

df.head(3)


,Car_id,Date,Customer Name,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price ($),Dealer_No,Body Style,Phone,Amount_paid_for_insurance,Claim amount,City
0,C_CND_000001,01/02/2022,Geraldine,Male,13500,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÃ‚Â Overhead Camshaft,Auto,Black,26000,06457-3834,SUV,8264678,1665,0.0,Riga
1,C_CND_000002,01/02/2022,Gia,Male,1480000,C & M Motors Inc,Dodge,Durango,DoubleÃ‚Â Overhead Camshaft,Auto,Black,19000,60504-7114,SUV,6848189,1332,1900.0,Liepaja
2,C_CND_000003,01/02/2022,Gianna,Male,1035000,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,31500,38701-8047,Passenger,7298798,1897,0.0,Riga


In [ ]:
#@title Make dataset for LagLlama

amounts = df[['Date', 'Claim amount']]

amounts['Date'] = pd.to_datetime(amounts['Date'])

# Set the date column as the index
amounts.set_index('Date', inplace=True)

# Aggregate daily claim amounts
amounts = amounts.resample('D').sum()

# Rename the columns
# amounts.reset_index(inplace=True)
# amounts.columns = ['ds', 'y']




<ipython-input-7-3d841cc413bb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amounts['Date'] = pd.to_datetime(amounts['Date'])


In [ ]:
#@title _
# # Group by date and summarize the claim amounts for each day
# daily_summary = amounts.groupby('Date')['Claim amount'].sum().reset_index()

# # Rename columns for clarity
# daily_summary.columns = ['Date', 'total_claim_amount']

# # Convert the 'Date' column to datetime
# daily_summary['Date'] = pd.to_datetime(daily_summary['Date'])

# # Set the date as the index
# daily_summary.set_index('Date', inplace=True)

# print(daily_summary)

# Set numerical columns as float32
# for col in daily_summary.columns:
#     # Check if column is not of string type
#     if daily_summary[col].dtype != 'object' and pd.api.types.is_string_dtype(daily_summary[col]) == False:
#         daily_summary[col] = daily_summary[col].astype('float32')

# print(daily_summary.dtypes)

# Create the Pandas
# dataset = PandasDataset.from_long_dataframe(
#     daily_summary,
#     target="total_claim_amount",
#     item_id="item_id",
#     freq='D'
#     )


In [ ]:
#@title LagLlama model init

backtest_dataset = _get_lag_llama_dataset(dataset=amounts)
print(backtest_dataset)
# backtest_dataset = daily_summary
prediction_length = 90  # Define your prediction length.
num_samples = 800 # number of samples sampled from the probability distribution for each timestep
device = torch.device("cpu") # You can switch this to CPU or other GPUs if you'd like, depending on your environment


PandasDataset<size=1, freq=D, num_feat_dynamic_real=0, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>


In [ ]:
#@title LagLlama model fit

# Make forecasts
forecasts, tss = get_lag_llama_predictions(
    backtest_dataset,
    prediction_length,
    device,
    num_samples,
    # context_length=32,
    use_rope_scaling=False
    )


In [ ]:
#@title LagLlama metrics

forecasts_ctx = list(forecasts)
tss_ctx = list(tss)

evaluator = Evaluator()
agg_metrics_ctx, ts_metrics_ctx = evaluator(iter(tss_ctx), iter(forecasts_ctx))


# print(agg_metrics_ctx)

print("MAE:", np.sqrt(agg_metrics_ctx['MSE']))

# # Calculate number of data points
# n = agg_metrics_ctx['abs_target_sum'] / agg_metrics_ctx['abs_target_mean']

# # Calculate SS_res
# SS_res = agg_metrics_ctx['MSE'] * n

# # Calculate SS_tot (total sum of squares)
# SS_tot = n * (agg_metrics_ctx['abs_target_mean'] ** 2)

# # Compute R²
# R2 = 1 - (SS_res / SS_tot)

# print(f"R²: {R2}")


Running evaluation: 1it [00:00, 17.18it/s]


MAE: 13089.01968317974


In [ ]:
#@title LagLlama plot matplotlib (disabled)

# plt.figure(figsize=(30, 18))
# date_formater = mdates.DateFormatter('%b, %d')
# plt.rcParams.update({'font.size': 15})

# # Iterate through the first 9 series, and plot the predicted samples
# for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
#     ax = plt.subplot(3, 3, idx+1)

#     plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
#     forecast.plot( color='r')
#     plt.xticks(rotation=60)
#     ax.xaxis.set_major_formatter(date_formater)
#     ax.set_title(forecast.item_id)

# plt.gcf().tight_layout()
# plt.legend()
# plt.show()


In [ ]:
#@title LagLlama forecast plot

forecast = forecasts[0]
ts = tss[0]

# Extract the samples
forecast_samples = forecast.samples

# Compute mean and percentiles
mean_forecast = np.mean(forecast_samples, axis=0)
percentile_2_5 = np.percentile(forecast_samples, 2.5, axis=0)
percentile_97_5 = np.percentile(forecast_samples, 97.5, axis=0)

# Convert forecast start date to string representation
forecast_start_date = str(forecast.start_date)

# Generate forecast dates
forecast_dates = pd.date_range(start=forecast_start_date, periods=len(mean_forecast))

# Define confidence interval trace
ci_trace = go.Scatter(
    x=np.concatenate([forecast_dates, forecast_dates[::-1]]),
    y=np.concatenate([percentile_2_5, percentile_97_5[::-1]]),
    fill='toself',
    fillcolor='rgba(255,0,0,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='95% Confidence Interval'
)

# Assuming prediction_length is defined somewhere in your code
prediction_length = 25  # for example

# Assuming you want to plot the last prediction_length periods
ts_trimmed = ts[-prediction_length:]

# Convert index of ts_trimmed to Timestamp
ts_trimmed_timestamp = ts_trimmed.index.to_timestamp()

# Concatenate training and test dataframes, with NaN between
ts_full = pd.concat([ts, pd.Series(index=[np.nan], data=[np.nan]), ts_trimmed])

# Convert index of ts_full to Timestamp
ts_full_timestamp = ts_full.index.to_timestamp()

# Create a figure
fig = go.Figure()

# Add trace for the full series (train + test)
fig.add_trace(go.Scatter(x=ts_full_timestamp, y=ts_full.values.flatten(), mode='lines', name='Full Series'))

# Add mean forecast trace
fig.add_trace(go.Scatter(x=forecast_dates, y=mean_forecast, mode='lines', name='Mean Forecast', line=dict(color='red', width=2)))

# Add confidence interval trace
fig.add_trace(ci_trace)

# Set x-axis date format
fig.update_xaxes(tickformat='%b %Y')

# Update layout
fig.update_layout(
    template='plotly_dark',
    plot_bgcolor='rgba(0, 0, 0, 1)',
    paper_bgcolor='rgba(0, 0, 0, 1)',
    title='Insurance Claims Forecast',
    xaxis_title='Date',
    yaxis_title='Daily Claim Amounts',
    legend_title='Legend',
    font=dict(size=15),
    # width=1800,
    # height=900
)

# Show the figure
pio.show(fig)


<ipython-input-19-b668f4ce05e1>:40: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [ ]:
#@title Make dataset for neural forecast


amounts_neural = df[['Date', 'Claim amount']]

amounts_neural['Date'] = pd.to_datetime(amounts_neural['Date'])

# Set the date column as the index
amounts_neural.set_index('Date', inplace=True)

# Aggregate daily claim amounts
amounts_neural = amounts_neural.resample('D').sum()

# Rename the columns
amounts_neural.reset_index(inplace=True)
amounts_neural.columns = ['ds', 'y']

# amounts_neural.head(10)


<ipython-input-14-dc8ed374cad3>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#@title Models fit neural forecast
%%capture

# Split data and declare panel dataset
Y_df = amounts_neural
Y_df['unique_id'] = 1.0
threshold_date = pd.to_datetime('2023-10-01')
Y_train_df = Y_df[Y_df.ds<=threshold_date] # 132 train
Y_test_df = Y_df[Y_df.ds>threshold_date] # 12 test

# Fit and predict with NBEATS and NHITS models
horizon = len(Y_test_df)
models = [NBEATS(
                input_size=2 * horizon,
                h=horizon,
                max_steps=500,
                ),
          NHITS(
              input_size=2 * horizon,
              h=horizon,
              max_steps=500
              ),
          LSTM(
              input_size=2 * horizon,
              h=horizon,                    # Forecast horizon
              max_steps=500,                # Number of steps to train
              # scaler_type='standard',       # Type of scaler to normalize data
              encoder_hidden_size=256,       # Defines the size of the hidden state of the LSTM
              decoder_hidden_size=256,
              ),     # Defines the number of hidden units of each layer of the MLP decoder
]
nf = NeuralForecast(models=models,
                    freq='D',
                    local_scaler_type='minmax'
                    )
nf.fit(df=Y_train_df)
Y_hat_df = nf.predict().reset_index()

# Clip predictions to be greater than 0
Y_hat_df['NBEATS'] = Y_hat_df['NBEATS'].clip(lower=0)
Y_hat_df['NHITS'] = Y_hat_df['NHITS'].clip(lower=0)
Y_hat_df['LSTM'] =Y_hat_df['LSTM'].clip(lower=0)


INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MAE           | 0     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     


In [ ]:
#@title Neural forecast plot

Y_hat_df = Y_test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])
plot_df = pd.concat([Y_train_df, Y_hat_df]).set_index('ds')


# Create traces for each line
trace_y = go.Scatter(x=plot_df.index, y=plot_df['y'], mode='lines', name='Actual', line=dict(width=2))
trace_nbeats = go.Scatter(x=plot_df.index, y=plot_df['NBEATS'], mode='lines', name='NBEATS', line=dict(width=2))
trace_nhits = go.Scatter(x=plot_df.index, y=plot_df['NHITS'], mode='lines', name='NHITS', line=dict(width=2))
trace_lstm = go.Scatter(x=plot_df.index, y=plot_df['LSTM'], mode='lines', name='LSTM', line=dict(width=2))

# Create layout
layout = go.Layout(
    template='plotly_dark',
    plot_bgcolor='rgba(0, 0, 0, 1)',
    paper_bgcolor='rgba(0, 0, 0, 1)',
    title='Insurance Claims Forecast',
    xaxis=dict(title='Date', tickfont=dict(size=14)),
    yaxis=dict(title='Daily Claim Amounts', tickfont=dict(size=14)),
    legend=dict(x=0.01, y=0.99, font=dict(size=12)),
    margin=dict(l=50, r=50, t=50, b=50),
)

# Combine traces into a single figure
fig = go.Figure(data=[trace_y, trace_nbeats, trace_nhits, trace_lstm], layout=layout)

# Show plot
fig.show()


In [ ]:
#@title Neural forecast metrics

# Calculate R² and MAE for train dataset
r2_test_nbeats = r2_score(Y_test_df['y'], Y_hat_df['NBEATS'])

mae_test_nbeats = mean_absolute_error(Y_test_df['y'], Y_hat_df['NBEATS'])

r2_test_nhits = r2_score(Y_test_df['y'], Y_hat_df['NHITS'])
mae_test_nhits = mean_absolute_error(Y_test_df['y'], Y_hat_df['NHITS'])

r2_test_lstm = r2_score(Y_test_df['y'], Y_hat_df['LSTM'])
mae_test_lstm = mean_absolute_error(Y_test_df['y'], Y_hat_df['LSTM'])

# Print the results
print("*"*40)
print(f"Test R² (NBEATS): {r2_test_nbeats}")
print(f"Test MAE (NBEATS): {mae_test_nbeats}")
print("*"*40)
print(f"Test R² (NHITS): {r2_test_nhits}")
print(f"Test MAE (NHITS): {mae_test_nhits}")
print("*"*40)
print(f"Test R² (LSTM): {r2_test_lstm}")
print(f"Test MAE (LSTM): {mae_test_lstm}")


****************************************
Test R² (NBEATS): -0.17106171008026672
Test MAE (NBEATS): 9888.680845323499
****************************************
Test R² (NHITS): -0.21237077070143862
Test MAE (NHITS): 9988.353863726605
****************************************
Test R² (LSTM): -0.4053984895364273
Test MAE (LSTM): 10280.92220893273


#### Based on the evaluation of the tested models, it is evident that NHITS consistently delivers superior results compared to the other models. Its performance in accurately forecasting the data stands out, making it the most effective model in this analysis.